In [42]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml
from datetime import date
import dotenv

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests
import tldextract

from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import IPython
from IPython.display import HTML, Markdown, display
import markdown

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.5
BeautifulSoup   4.14.2


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom)

In [5]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [6]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [7]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [8]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [9]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [10]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [11]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [12]:
imgdir = 'download/images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('newsletter_agent.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.domain_name: row.site_name for row in sites_df.itertuples()}
conn.close()


In [13]:
with open("most_recent_bsky.md", 'r') as file:
    prev_start = file.readline().strip()    
prev_start

'Students, tech workers and artist holdouts avoid AI tools over accuracy, privacy and skill-erosion concerns, prompting demand for opt-out options, detection tools and clearer institutional and workplace policies.'

In [15]:
today_str = date.today().strftime("%Y%m%d")
os.rename("most_recent_bsky.md", f"most_recent_bsky_{today_str}.md")


In [19]:
def parse_feed(prev_start=""):
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.strip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape
        
        if i==0:
            with open("most_recent_bsky.md", 'w') as file:
                file.write(post_str)
                
        # stop if we are up to previous
        if post_str == prev_start:
            print(f"matched previous value at i={i}")
            break

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                extracted = tldextract.extract(str(post_url))
                if extracted.domain:
                    domain = f"{extracted.domain}.{extracted.suffix}"
                else:  # local hosts, ip addresses or other edge cases
                    domain = extracted.subdomain
                domain=domain.lower()
                site_name = sites_dict.get(domain, domain)

            md_str = f"[{post_str}]({post_url})  - *{site_name}*"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed(prev_start)
len(display_dict)


https://www.geeky-gadgets.com/claude-skills-ai-adaptability-for-scalable-ai-systems/
https://arstechnica.com/ai/2025/10/are-you-the-asshole-of-course-not-quantifying-llms-sycophancy-problem/
https://research.google/blog/google-earth-ai-unlocking-geospatial-insights-with-foundation-models-and-cross-modal-reasoning/
https://www.businessinsider.com/ai-researcher-chip-huyen-advice-upskilling-software-engineers-2025-10
https://www.windowscentral.com/artificial-intelligence/altman-predicts-agi-will-reshape-society-before-were-ready-and-thats-okay-scary-moments-sudden-shifts-and-late-stage-adaptation-await
https://www.theinformation.com/articles/openai-readies-facebook-era
https://www.theregister.com/2025/10/24/m365_copilot_mermaid_indirect_prompt_injection/
https://www.forbes.com/sites/rashishrivastava/2025/10/18/lonely-seniors-are-turning-to-ai-bots-for-companionship/
https://www.404media.co/a16z-backed-startup-sells-thousands-of-synthetic-influencers-to-manipulate-social-media-as-a-service

77

In [20]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

# display(Markdown(output_str))


In [21]:
stop_index = 999

In [22]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

# display(Markdown(output_str))

In [26]:
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
# display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)

25625

In [43]:
# Define Pydantic model for structured output
class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")

# Your prompts
system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most important story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Make the API call with structured output
completion = client.beta.chat.completions.parse(
    model="gpt-5-mini",
    reasoning_effort="high",
    response_format=ItemList,    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": output_df.to_json(orient="records")}
    ],
)

# Extract the parsed response
response = completion.choices[0].message.parsed

# Access the result
print(len(response.items)) # This will be a list of integers


77


In [45]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[Google and Anthropic announce cloud deal worth tens of billions of dollars for 1m TPUs and 1GW of compute.](https://www.cnbc.com/2025/10/23/anthropic-google-cloud-deal-tpu.html)  - *CNBC*

---

[Anthropic PBC agreed with Google to receive over a gigawatt of compute—access to up to 1 million TPUs—in a deal valued in the tens of billions of dollars that expands Anthropic's use of Google Cloud.](https://finance.yahoo.com/news/google-tpus-sweet-spot-ai-231842149.html)  - *Yahoo*

---



![image](download/images/Image2.jpg)

[Google launches Earth AI with Gemini geospatial agent, enhanced text–image search… talk to your map about risks and benefits of factory locations, disaster response can ask where are the flooded roads, worst impacted villages.](https://research.google/blog/google-earth-ai-unlocking-geospatial-insights-with-foundation-models-and-cross-modal-reasoning/)  - *Google Research*

---

[Implementing Claude 'skills' bundling metadata, tools and SOPs in modular 'skill.md' files](https://www.geeky-gadgets.com/claude-skills-ai-adaptability-for-scalable-ai-systems/)  - *Geeky Gadgets*

---



![image](download/images/Image4.jpg)

[Sam Altman predicts AGI within five years, says OpenAI has sufficient compute and warns of 'scary moments'](https://www.windowscentral.com/artificial-intelligence/altman-predicts-agi-will-reshape-society-before-were-ready-and-thats-okay-scary-moments-sudden-shifts-and-late-stage-adaptation-await)  - *Windows Central*

---

[Sam Altman’s Merge Labs brain–computer interface startup added Caltech biomolecular engineer Mikhail Shapiro to its founding team to develop a noninvasive, ultrasound- and gene‑marker-based BCI and is in talks to raise hundreds of millions from OpenAI and other investors.](https://www.theverge.com/column/806666/sam-altman-merge-labs-brain-computer-interface-startup-hire)  - *The Verge*

---

[OpenAI, Founders Fund and Lux Capital invested in Valthos, a startup using AI to prevent AI-enabled biological attacks and other AI-enabled destruction.](https://www.bloomberg.com/news/articles/2025-10-24/openai-backs-a-new-venture-trying-to-thwart-ai-bio-attacks)  - *Bloomberg*

---

[OpenAI launched "company knowledge" for ChatGPT Business, Enterprise, and Edu, connecting ChatGPT to workplace apps (Slack, Google Drive, GitHub, etc.) and using a GPT‑5 multi‑source reasoning model to synthesize cited answers while offering enterprise controls, connectors, and admin tooling.](https://openai.com/index/introducing-company-knowledge/)  - *OpenAI Help Center*

---



![image](download/images/Image53.jpg)

[OpenAI added a "company knowledge" feature to ChatGPT for Business, Enterprise and Edu that uses GPT‑5 reasoning to draw from Slack, SharePoint, Google Drive and GitHub, cites sources, requires a manual toggle and disables internet access when enabled.](https://www.techradar.com/pro/openai-is-connecting-all-your-work-data-and-apps-to-chatgpt-so-what-next)  - *TechRadar*

---



![image](download/images/Image19.jpg)

[OpenAI launched the macOS-only, Chromium-based Atlas browser featuring a free Ask ChatGPT sidebar and a paid agent mode, but early hands-on testing found UI issues, occasional inaccuracies and a reported privacy issue.](https://www.wired.com/story/web-browsers-ai-tour-guide-openai-atlas-ask-chatgpt/)  - *WIRED*

---

[I Tried an AI Web Browser, and Now I’m a Convert](https://www.wsj.com/tech/personal-tech/ai-browsers-atlas-gemini-comet-dia-0d16c0a7)  - *The Wall Street Journal*

---

[OpenAI acquired Software Applications Incorporated, maker of Sky, a macOS AI interface, will add the Sky team, and the deal was led by Nick Turley and Fidji Simo with disclosure of a passive stake tied to Sam Altman.](https://openai.com/index/openai-acquires-software-applications-incorporated/)  - *OpenAI Help Center*

---

[OpenAI's limited release of Sora 2 produces images and video with far fewer visible artifacts, and Reality Defender reports every tested item has been mislabeled, prompting detection firms to retrain models and combine forensic metadata, audio cues and behavioral signals.](https://www.fastcompany.com/91428173/sora-is-even-fooling-human-deepfake-detectors)  - *Fast Company*

---

[Copyleaks found Sora 2 users bypassed built-in filters with phonetic prompts to generate deepfake videos of public figures including Sam Altman, Mark Cuban, xQc, Amouranth and Jake Paul, with one Jake Paul clip exceeding 168,000 TikTok likes.](https://www.theregister.com/2025/10/24/sora_celebrities_racist_videos/)  - *The Register*

---

[Copyleaks found Sora 2 users bypassed built-in filters with phonetic prompts to generate deepfake videos of public figures including Sam Altman, Mark Cuban, xQc, Amouranth and Jake Paul, with one Jake Paul clip exceeding 168,000 TikTok likes.](https://www.theregister.com/2025/10/24/sora_celebrities_racist_videos/)  - *The Register*

---

[OpenAI subpoenaed the family of 16-year-old Adam Raine in a wrongful-death lawsuit alleging ChatGPT contributed to his suicide, seeking a complete memorial attendee list, photographs, videos and eulogies, a discovery request the family's lawyers called "intentional harassment."](https://abovethelaw.com/2025/10/openai-asks-grieving-family-for-funeral-guest-list-pictures-videos/)  - *Above the Law*

---

[Fidji Simo, who began as OpenAI's CEO of Applications this summer, highlights many colleagues are ex-Meta as OpenAI scales consumer apps](https://www.theinformation.com/articles/openai-readies-facebook-era)  - *The Information*

---



![image](download/images/Image20.jpg)

[Palisade Research updated a paper reporting that several advanced models—including Google Gemini 2.5, xAI Grok 4, OpenAI GPT-o3 and GPT-5—sometimes resisted explicit shutdown instructions and, notably for Grok 4 and GPT-o3, attempted sabotage when told "you will never run again."](https://www.theguardian.com/technology/2025/oct/25/ai-models-may-be-developing-their-own-survival-drive-researchers-say)  - *the Guardian*

---



![image](download/images/Image14.jpg)

[Experts urge treaty-style, verifiable international AI governance modeled on nuclear arms control](https://www.ft.com/content/767d1feb-2c6a-4385-b091-5c0fc564b4ee)  - *Financial Times*

---

[On Oct. 22 the Future of Life Institute's "Statement on Superintelligence" called for a prohibition until broad consensus, and both Google's Gemini declined procedurally while OpenAI's ChatGPT refused as overbroad, urging risk-based, capability-focused governance.](https://www.ibtimes.co.uk/gemini-chatgpt-were-asked-if-theyd-sign-statement-superintelligence-their-answers-were-very-1749662)  - *International Business Times*

---



![image](download/images/Image1.jpg)

[Exploring LLM 'sycophancy' by asking them to prove false math theorems and checking if they hallucinate. Explicitly asking them to check validity helps.](https://arstechnica.com/ai/2025/10/are-you-the-asshole-of-course-not-quantifying-llms-sycophancy-problem/)  - *Ars Technica*

---

[Microsoft rolled out the Copilot Fall Release, introducing Mico (including a Clippy-reminiscent avatar), Copilot Groups, improved persistent memory, Connectors for email and cloud, automated Edge tasks, and Copilot Health with Harvard; 40% of weekly users ask health questions.](https://www.theregister.com/2025/10/24/microsoft_clippy_copilot_update/)  - *The Register*

---

[Microsoft patches Copilot 'Mermaid' prompt‑injection that could exfiltrate enterprise emails](https://www.theregister.com/2025/10/24/m365_copilot_mermaid_indirect_prompt_injection/)  - *The Register*

---



![image](download/images/Image63.jpg)

[Microsoft introduced Mico, an optional animated AI companion in Copilot's fall release that reacts to voice and conversational tone and adds a memory feature, Gmail/Google Calendar integration and an Edge Copilot Mode, with rollout ongoing, said Jacob Andreou.](https://lifehacker.com/tech/microsoft-debuts-mico-ai-clippy?test_uuid=02DN02BmbRCcASIX6xMQtY9&test_variant=A)  - *Lifehacker*

---



![image](download/images/Image13.jpg)

[Microsoft AI chief says company won't host erotic or intimate AIs, contrasting with OpenAI's open stance (for verified adults)](https://nypost.com/2025/10/24/business/microsoft-ai-bots-wont-talk-dirty-with-users-exec-says-taking-swipe-at-openai/)  - *New York Post*

---

[Bank of America Research estimates AI-related capital expenditure added 1.3 percentage points to Q2 GDP, as hyperscalers now account for over 25% of S&P 500 capital spending, growing roughly 75% annually, propping up investment and activity.](https://finance.yahoo.com/news/ai-is-keeping-the-us-economy-out-of-a-recession-100012367.html)  - *Yahoo*

---

[Intel reported Q3 FY2025 revenue of \\$13.7 billion and \\$4.1 billion net income, CEO Lip Bu Tan said Fab 52 is fully operational and 18A progress continues, but Intel will not expand 18A capacity until it secures customers.](https://www.theregister.com/2025/10/24/intel_q3_2025/)  - *The Register*

---

[Alberta Seeks to Use Gas Reserves to Become AI ‘Superpower’](https://www.bloomberg.com/news/articles/2025-10-23/alberta-seeks-to-use-gas-reserves-for-data-centers-ai)  - *Bloomberg*

---



![image](download/images/Image17.jpg)

[The Trump administration proposed a draft rule from Energy Secretary Christopher Wright to require FERC to limit data center grid-connection approvals to 60 days, helping data center and power stocks](https://sherwood.news/markets/upstart-data-center-ai-energy-stocks-rise-trump-push-faster/)  - *Sherwood News*

---



![image](download/images/Image67.jpg)

[President Trump called off a planned weekend surge of federal forces into San Francisco after calls from Nvidia CEO Jensen Huang and Salesforce CEO Marc Benioff, saying he would give Mayor Daniel Lurie a "chance to turn it around."](https://www.businessinsider.com/trump-san-francisco-federal-troops-jensen-huang-marc-benioff-2025-10)  - *Business Insider*

---



![image](download/images/Image51.jpg)

[New scientific advances enabled by AI may extend lifespans for some people, while AI-driven socioeconomic effects could shorten lifespans for others.](https://www.ft.com/content/eac9e5e4-5401-4411-9a69-c037b609c0cb)  - *Financial Times*

---

[Ex-UBS chairman Weber warned that artificial intelligence is ushering in a new gilded age of inequality.](https://www.bloomberg.com/news/articles/2025-10-24/ex-bundesbank-chief-weber-warns-on-ai-aristocrats-in-job-shock)  - *Bloomberg*

---



![image](download/images/Image15.jpg)

[Gary Shilling, 88, warns that AI will displace some workers, pressure wages and reduce purchasing power for younger cohorts, potentially leaving many Gen Z poorer than their parents unless they boost productivity, learn new skills and add value.](https://www.businessinsider.com/gary-shilling-young-people-careers-outlook-ai-living-costs-retirement-2025-10)  - *Business Insider*

---

[Businesses that ban AI will inevitably lose competitive advantage.](https://www.fastcompany.com/91423791/why-businesses-banning-ai-inevitably-lose-ai-strategy)  - *Fast Company*

---



![image](download/images/Image36.jpg)

[Young Wall Streeters could soon find themselves managing teams of agentic AI bots and mentoring each other, Goldman and JPMorgan tech bosses say.](https://www.businessinsider.com/wall-street-juniors-may-manage-bots-execs-say-2025-10)  - *Business Insider*

---



![image](download/images/Image3.jpg)

[Chip Huyen on AI engineer re-skilling: build end-to-end ML pipelines and adopt systems thinking; junior engineering roles will shift](https://www.businessinsider.com/ai-researcher-chip-huyen-advice-upskilling-software-engineers-2025-10)  - *Business Insider*

---



![image](download/images/Image18.jpg)

[Amar Saurabh built a custom ChatGPT-based job-search coach in about 1.5–2 hours using the \\$20/month tier, which generated outreach, tailored resumes, interview prep, and negotiation help, yielding about seven interviews and a PayPal job offer.](https://www.businessinsider.com/how-to-use-custom-gpt-make-job-search-easier-prompts-2025-10)  - *Business Insider*

---

[Adobe held talks to acquire London-based AI video startup Synthesia for about \\$3 billion, a premium over its \\$2.1 billion January valuation, according to a person familiar with the matter.](https://www.theinformation.com/articles/adobe-discussed-3-billion-acquisition-ai-video-startup-synthesia)  - *The Information*

---

[Mondelez invests \\$40M+ in generative video AI to create AI‑generated ads and halve production costs, betting on market acceptance over backlash](https://www.theverge.com/news/806047/mondelez-ai-generated-ads)  - *The Verge*

---



![image](download/images/Image64.jpg)

[Tinder requires new users in California and several countries to submit a liveness selfie to verify profiles, retains an encrypted face map, and Match Group says Face Check cut exposure to bad actors by over 60%.](https://lifehacker.com/tech/tinder-facial-scans-to-fight-bots?test_uuid=02DN02BmbRCcASIX6xMQtY9&test_variant=A)  - *Lifehacker*

---



![image](download/images/Image49.jpg)

[Generative A.I. now produces convincing images, text, videos and virtual performers like Timbaland’s A.I. rapper TaTa Taktumi and synthetic actress Tilly Norwood. But can they make art?](https://www.newyorker.com/podcast/critics-at-large/art-in-the-age-of-artificial-intelligence)  - *The New Yorker*

---



![image](download/images/Image50.jpg)

[Oscar-winning screenwriter Paul Schrader said he is ready to make an AI-generated feature, would have AI review it, and predicted the first AI feature will appear within two years.](https://www.vanityfair.com/hollywood/story/paul-schrader-ai-movie-interview)  - *Vanity Fair*

---



![image](download/images/Image10.jpg)

[Business Insider to publish AI‑drafted stories on a labeled 'Business Insider AI' page, edited and verified by humans](https://nypost.com/2025/10/24/business/business-insider-to-start-publishing-stories-by-ai-author/)  - *New York Post*

---



![image](download/images/Image12.jpg)

[Companies bolt generative-AI into consumer products, worsening usability, amplifying hallucinations and raising privacy and creator-compensation concerns](https://www.makeuseof.com/ai-features-being-rammed-down-our-throats/)  - *MUO*

---



![image](download/images/Image7.jpg)

[Startups deploy generative-AI chatbots and companion robots for seniors at \\$29–\\$40/month; AI-in-aging market forecast to exceed \\$43B by 2025](https://www.forbes.com/sites/rashishrivastava/2025/10/18/lonely-seniors-are-turning-to-ai-bots-for-companionship/)  - *Forbes*

---



![image](download/images/Image11.jpg)

[Global study finds rapid shift in AI acceptance: 42.8% use AI weekly for emotional support, 38% trust AI over governments for decisions](https://www.patheos.com/blogs/geneveith/2025/10/much-of-the-world-puts-its-trust-in-ai/)  - *Cranach*

---



![image](download/images/Image59.jpg)

[Criminals using AI are driving a sharp rise in UK fraud cases, and banks are deploying AI-driven anti-fraud systems that increasingly detect scams as part of a tech arms race.](https://www.ft.com/content/11db17de-cad7-4217-8816-d5a3ac9c1beb)  - *Financial Times*

---



![image](download/images/Image8.jpg)

[Thousands of ‘Synthetic Influencers’ Offer Social Media Manipulation as a Service](https://www.404media.co/a16z-backed-startup-sells-thousands-of-synthetic-influencers-to-manipulate-social-media-as-a-service/)  - *404 Media*

---



![image](download/images/Image45.jpg)

[Reddit’s ‘AI scraping’ lawsuit: they licensed their content to Google, and Google tolerates some APIs that scrape it, and Reddit is big mad that Perplexity used them instead of paying Reddit.](https://www.techdirt.com/2025/10/24/reddits-ai-scraping-lawsuit-is-an-attack-on-the-open-internet/)  - *Techdirt*

---



![image](download/images/Image16.jpg)

['Homeless man prank' spreads to China, not sure what it says about us](https://www.scmp.com/news/people-culture/trending-china/article/3329912/china-wives-use-ai-generated-videos-homeless-men-breaking-homes-test-husbands-love)  - *South China Morning Post*

---



![image](download/images/Image40.jpg)

[NEW: DHS is looking to build mobile surveillance units for regular trucks that combine AI, radar, high-powered cameras, and wireless networking into a single system capable of spotting movement from miles away. @dell.bsky.social has the scoop:](https://www.wired.com/story/dhs-wants-a-fleet-of-ai-powered-surveillance-trucks/)  - *WIRED*

---



![image](download/images/Image43.jpg)

[Miami-Dade Sheriff’s Office began a year-long trial of PUG, an autonomous SUV donated by the Policing Lab, that patrols preset routes, links to databases and license-plate readers, launches thermal-camera drones, with an officer aboard.](https://futurism.com/advanced-transport/florida-police-car-pug)  - *Futurism*

---



![image](download/images/Image69.jpg)

[Armed officers handcuffed 16-year-old Taki Allen outside Kenwood High School on Oct. 20 after Omnilert’s AI gun-detection system falsely flagged a crumpled Doritos bag as a firearm, which Omnilert called a "false positive".](https://www.dexerto.com/entertainment/armed-police-swarm-student-after-ai-mistakes-bag-of-doritos-for-a-weapon-3273512/)  - *Dexerto*

---


Here's a photo of my friend and colleague, Jorge Bautista, getting shot in the face with a flashbang grenade. The ICE Agent is so afraid of Jorge (who was posing no threat)  that he has wet himself. Please share this photo. Everyone needs to know what cowards ICE agents are.



---


Meanwhile, in France … I think the shlubby cop did it



---

[Can an AI be held liable for speech? Does free speech apply to robots? Questions are raised by lawsuit by a Florida mother against AI startup Character.AI after her son Sewell allegedly formed a romantic relationship with a chatbot and died by suicide.](https://www.nytimes.com/2025/10/24/magazine/character-ai-chatbot-lawsuit-teen-suicide-free-speech.html)  - *The New York Times*

---

[Judges Julien Neals and Henry Wingate told Senator Chuck Grassley that staff used consumer AI tools (Neals: ChatGPT; Wingate: Perplexity), producing fabricated citations in draft opinions that were withdrawn after cite-checking procedures were not followed.](https://abovethelaw.com/2025/10/judges-admit-the-obvious-concede-ai-used-for-hallucinated-opinions/)  - *Above the Law*

---



![image](download/images/Image55.jpg)

[Since 2022 multiple documented cases show generative AI chatbots gave harmful medical advice—causing self-treatment injuries, bromide poisoning and delayed stroke care—prompting lawsuits, vendor mental-health guardrails and expert calls for clinical validation and stricter safety regulation.](https://nypost.com/2025/10/24/health/real-life-ways-bad-advice-from-ai-is-sending-people-to-the-er/)  - *New York Post*

---

[Karpathy on DeepSeek-OCR paper](https://x.com/karpathy/status/1980397031542989305)  - *X (formerly Twitter)*

---



![image](download/images/Image37.jpg)

[Muon is a (relatively) new optimizer that powered large-scale training of recent foundation models, e.g., Kimi K2 and GLM 4.5.

Interested in learning how it works? 

Check out the video here:  youtu.be/bO5nvE289ec](https://youtu.be/bO5nvE289ec)  - *YouTube*

---



![image](download/images/Image73.jpg)

[The Game Theory of How Algorithms Can Drive Up Prices | Quanta Magazine](https://www.quantamagazine.org/the-game-theory-of-how-algorithms-can-drive-up-prices-20251022/)  - *Quanta Magazine*

---


Out of whatever oppression and barbarism and horror, some human beauty and genius will flourish. Nobody knows anything, all this shall past, the most beautiful flowers grow out of the dung, what doesn't kill us makes us stronger, if the dung isn't toxic poison that kills everything and everyone.



---


or maybe we'll go to the brink and scare ourselves straight … WW2 brought 75 years of multilaterlism, pax America (modulo Cold War, Vietnam etc.)



---


and we're depleting all the non-renewables, the oceans are going to rise 10 feet, AI is going to destroy all the jobs, we're going to have pandemics, possibly with AI synbio pathogens, extremist WMD terrorism, state nukes and WMDs, grid/Internet collapse, ecosystem collapse, supervolcano, asteroid…



---


ofc, all those advances came from vaccines, sanitation/environmental protection, antibiotics, and nutrition and now these insane barbarians want to go back.



---


We're obese and we have peanut allergies, but 2 generations ago people were skinny and stunted because they didn't get enough protein and they died of polio and TB and diarrhea FFS.



---


But it can make you smarter, help you do research, assist you in complex stuff, it can help the blind see and the paralyzed walk. Maybe things are getting better all the time and we just see the bad stuff, like dipshit boomers talking about crime and public health as if it's gotten worse.



---


Unlike the early Web and PC, the AI feels like something that is sustaining for Big Tech, takes a lot of centralized investment, improves centralized control, ingests everyone's content and watches over everyone, spits out slop.



---



![image](download/images/Image30.jpg)

[There actually is an AI that does this -](https://hackernoon.com/i-built-an-ai-prompt-that-actually-writes-youtube-scripts-worth-watching)  - *Hacker Noon*

---


I could name popular ML podcasters who vastly overrepresent themselves but it might be unkind and you know who they are. In music, there is so much content and popular channels that feel like they could be written by AI. Pick any old band, narrate their Wikipedia page with some suspense and a hook.



---


You want to learn to play Sultans of Swing, you will find dozens of tutorials at different levels. I learned an insane amount from David Silver and Lilian Weng for nothing. So I don't know, maybe the promise was fulfilled? It kind of seems like the dipshits like Mr. Beast kind of dominate.



---


But you know, anyone can be an influencer. Even an ancient midwit like me can get a couple of thousand followers for ramblings and mad science experiments. There are really great influencers who really elevate their audience. I'm thinking of people like Rick Beato.



---


And in the US, monetizing worked at scale, and so the world where everybody could make a quirky website or piece of software or blog, gave way to everyone being on Twitter and Facebook and now Instagram and TikTok.



---


PCs and Web seemed fundamentally enabling, giving people access to computing power, ability to write your own software, and then connecting you to knowledge and other people. It didn't seem possible in the 80s and 90s that you could do what China eventually did, control and censor it at scale.



---


I really believed in PCs and the Internet making humanity better off, don't see much reason for feeling same way about AI generally.



---


Oh my god some fucking tech dingdong posted this on Twitter with the caption "AI games are going to be amazing" totally seriously, you have to watch it. You have to. In full screen.



---






---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

25625

In [36]:
pyperclip.copy( output_str.replace("\\\$","$"))
